# Multiclass task

## Initial phase

### Import necessary libraries 

In [1]:
# Standard python librari### Import necessary libraries es
import logging
import os
import time
logging.basicConfig(format='[%(asctime)s] (%(levelname)s): %(message)s', level=logging.INFO)

# Installed libraries
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.model_selection import train_test_split
import torch

# Imports from our package
from lightautoml.automl.base import AutoML
from lightautoml.automl.blend import WeightedBlender
from lightautoml.ml_algo.boost_lgbm import BoostLGBM
from lightautoml.ml_algo.linear_sklearn import LinearLBFGS
from lightautoml.ml_algo.tuning.optuna import OptunaTuner
from lightautoml.pipelines.features.lgb_pipeline import LGBSimpleFeatures, LGBAdvancedPipeline
from lightautoml.pipelines.features.linear_pipeline import LinearFeatures
from lightautoml.pipelines.ml.base import MLPipeline
from lightautoml.pipelines.selection.importance_based import ModelBasedImportanceEstimator, ImportanceCutoffSelector
from lightautoml.reader.base import PandasToPandasReader
from lightautoml.tasks import Task
from lightautoml.utils.timer import PipelineTimer

### Parameters 

In [2]:
### Parameters N_THREADS = 8 # threads cnt for lgbm and linear models
N_FOLDS = 5 # folds cnt for AutoML
RANDOM_STATE = 42 # fixed random state for various reasons
TEST_SIZE = 0.2 # Test size for metric check
TIMEOUT = 600 # Time in seconds for automl run
TARGET_NAME = 'TARGET' # Target column name
N_THREADS = 8 # Nubmer ot threads

###  Fix torch number of threads and numpy seed 

In [3]:
np.random.seed(RANDOM_STATE)
torch.set_num_threads(N_THREADS)

### Example data load 

In [5]:
%%time

data = pd.read_csv('../LightAutoML/examples/data/sampled_app_train.csv')
data.head()

CPU times: user 96.6 ms, sys: 28.2 ms, total: 125 ms
Wall time: 124 ms


,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,313802,0,Cash loans,M,N,Y,0,270000.0,327024.0,15372.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,319656,0,Cash loans,F,N,N,0,108000.0,675000.0,19737.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,207678,0,Revolving loans,F,Y,Y,2,112500.0,270000.0,13500.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
3,381593,0,Cash loans,F,N,N,1,67500.0,142200.0,9630.0,...,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
4,258153,0,Cash loans,F,Y,Y,0,337500.0,1483231.5,46570.5,...,0,0,0,0,0.0,0.0,0.0,2.0,0.0,0.0


### (Optional) Some user feature preparation 

Cell below shows some user feature preparations to create task more difficult (this block can be omitted if you don't want to change the initial data):

In [6]:
%%time

data['BIRTH_DATE'] = (np.datetime64('2018-01-01') + data['DAYS_BIRTH'].astype(np.dtype('timedelta64[D]'))).astype(str)
data['EMP_DATE'] = (np.datetime64('2018-01-01') + np.clip(data['DAYS_EMPLOYED'], None, 0).astype(np.dtype('timedelta64[D]'))
                    ).astype(str)

data['constant'] = 1
data['allnan'] = np.nan

data['report_dt'] = np.datetime64('2018-01-01')

data.drop(['DAYS_BIRTH', 'DAYS_EMPLOYED'], axis=1, inplace=True)

CPU times: user 105 ms, sys: 8.13 ms, total: 113 ms
Wall time: 112 ms


### Create fake multiclass target 

In [7]:
data[TARGET_NAME] = np.where(np.random.rand(data.shape[0]) > .5, 2, data[TARGET_NAME].values)
data[TARGET_NAME].value_counts()

2    4924
0    4677
1     399
Name: TARGET, dtype: int64

### (Optional) Data splitting for train-test 

Block below can be omitted if you are going to train model only or you have specific train and test files:

In [8]:
%%time

train_data, test_data = train_test_split(data, 
                                         test_size=TEST_SIZE, 
                                         stratify=data[TARGET_NAME], 
                                         random_state=RANDOM_STATE)
logging.info('Data splitted. Parts sizes: train_data = {}, test_data = {}'
              .format(train_data.shape, test_data.shape))

[2020-12-03 18:21:48,824] (INFO): Data splitted. Parts sizes: train_data = (8000, 125), test_data = (2000, 125)


CPU times: user 10.5 ms, sys: 3.17 ms, total: 13.6 ms
Wall time: 12.8 ms


In [9]:
train_data.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,...,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,BIRTH_DATE,EMP_DATE,constant,allnan,report_dt
3273,293399,2,Revolving loans,F,Y,N,0,292500.0,765000.0,38250.0,...,0.0,0.0,0.0,0.0,0.0,1980-08-03,2013-06-26,1,NaN,2018-01-01
812,426509,1,Cash loans,F,N,Y,1,225000.0,349258.5,17959.5,...,0.0,0.0,0.0,0.0,5.0,1984-05-06,2013-02-17,1,NaN,2018-01-01
9630,403114,0,Cash loans,M,Y,Y,2,135000.0,922500.0,29889.0,...,0.0,0.0,4.0,0.0,4.0,1984-01-23,2009-06-01,1,NaN,2018-01-01
7757,366682,2,Cash loans,F,N,N,0,254475.0,1485000.0,39303.0,...,0.0,0.0,0.0,0.0,0.0,1971-11-28,2016-01-10,1,NaN,2018-01-01
9973,289050,0,Cash loans,F,N,Y,0,112500.0,260640.0,25906.5,...,NaN,NaN,NaN,NaN,NaN,1966-11-27,2002-07-12,1,NaN,2018-01-01


## AutoML Creation

![AutoML pipeline for this task](imgs/tutorial_2_pipeline.png)


### Create Timer for pipeline

Here we are going to use strict timer for AutoML pipeline, which helps not to go outside the limit:

In [10]:
%%time

timer = PipelineTimer(600, mode=2)

CPU times: user 149 µs, sys: 87 µs, total: 236 µs
Wall time: 241 µs


### Create feature selector

In [11]:
%%time

timer_gbm = timer.get_task_timer('gbm') # Get task timer from pipeline timer 
feat_sel_0 = LGBSimpleFeatures()
mod_sel_0 = BoostLGBM(timer=timer_gbm)
imp_sel_0 = ModelBasedImportanceEstimator()
selector_0 = ImportanceCutoffSelector(feat_sel_0, mod_sel_0, imp_sel_0, cutoff=0, )

Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer


CPU times: user 2.24 ms, sys: 0 ns, total: 2.24 ms
Wall time: 1.87 ms


### Create GBMs pipeline for AutoML 

Our GBMs ML pipeline:
- Advanced features for gradient boosting built on selected features (using step 2) 
- 2 different models:
    * LightGBM with params tuning (using OptunaTuner)
    * LightGBM with heuristic params


In [12]:
%%time 

feats_gbm_0 = LGBAdvancedPipeline(top_intersections=4, 
                                  output_categories=True, 
                                  feats_imp=imp_sel_0)
timer_gbm_0 = timer.get_task_timer('gbm')
timer_gbm_1 = timer.get_task_timer('gbm')

gbm_0 = BoostLGBM(timer=timer_gbm_0)
gbm_1 = BoostLGBM(timer=timer_gbm_1)

tuner_0 = OptunaTuner(n_trials=20, timeout=30, fit_on_holdout=True)
gbm_lvl0 = MLPipeline([
        (gbm_0, tuner_0),
        gbm_1
    ],
    pre_selection=selector_0,
    features_pipeline=feats_gbm_0, 
    post_selection=None
)

CPU times: user 1.22 ms, sys: 55 µs, total: 1.28 ms
Wall time: 1.28 ms


### Create linear pipeline for AutoML 

Our linear pipeline:
- Using features, special for linear models
- LinearLBFGS as a model
- Without feature selection here

In [13]:
%%time

feats_reg_0 = LinearFeatures(output_categories=True, 
                             sparse_ohe='auto')

timer_reg = timer.get_task_timer('reg')
reg_0 = LinearLBFGS(timer=timer_reg)

reg_lvl0 = MLPipeline([
        reg_0
    ],
    pre_selection=None,
    features_pipeline=feats_reg_0, 
    post_selection=None
)

CPU times: user 547 µs, sys: 318 µs, total: 865 µs
Wall time: 868 µs


### Create multiclass task and reader

In [14]:
%%time 

task = Task('multiclass', metric = 'crossentropy', ) 
reader = PandasToPandasReader(task = task, samples = None, max_nan_rate = 1, max_constant_rate = 1,
                              advanced_roles = True, drop_score_co = -1, n_jobs = 4)

CPU times: user 4.99 ms, sys: 564 µs, total: 5.56 ms
Wall time: 4.67 ms


### Create blender for 2nd level 

To combine predictions from different models into one vector we use WeightedBlender:

In [15]:
%%time

blender = WeightedBlender()

CPU times: user 103 µs, sys: 8 µs, total: 111 µs
Wall time: 115 µs


### Create AutoML pipeline

In [16]:
%%time

automl = AutoML(reader=reader, levels=[
    [gbm_lvl0, reg_lvl0]
], timer=timer, blender=blender, skip_conn=False, verbose=0)

CPU times: user 669 µs, sys: 0 ns, total: 669 µs
Wall time: 673 µs


### Train AutoML on loaded data 

In cell below we train AutoML with target column `TARGET` to receive fitted model and OOF predictions:

In [17]:
%%time

oof_pred = automl.fit_predict(train_data, roles={'target': TARGET_NAME})
logging.info('oof_pred:\n{}\nShape = {}'.format(oof_pred, oof_pred.shape))

Train data shape: (8000, 125)
Feats was rejected during automatic roles guess: []
Start fitting LightGBM ...
Training until validation scores don't improve for 100 rounds
[100]	valid's multi_logloss: 0.833296
Early stopping, best iteration is:
[32]	valid's multi_logloss: 0.821232
LightGBM fitting and predicting completed
Optuna may run 162.65567008654276 secs


[2020-12-03 18:21:59,996] (INFO): A new study created in memory with name: no-name-ea8e74a6-5e87-4df3-8f06-af1ab1d4d340


Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
Training until validation scores don't improve for 100 rounds
[100]	valid's multi_logloss: 0.82247
Early stopping, best iteration is:
[55]	valid's multi_logloss: 0.816465
Lvl_0_Pipe_0_Mod_0_LightGBM fitting and predicting completed


[2020-12-03 18:22:03,949] (INFO): Trial 0 finished with value: -0.8164646521955728 and parameters: {'feature_fraction': 0.6872700594236812, 'num_leaves': 108}. Best is trial 0 with value: -0.8164646521955728.


Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
Training until validation scores don't improve for 100 rounds
[100]	valid's multi_logloss: 0.821377
Early stopping, best iteration is:
[52]	valid's multi_logloss: 0.815787
Lvl_0_Pipe_0_Mod_0_LightGBM fitting and predicting completed


...

Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
Training until validation scores don't improve for 100 rounds
[100]	valid's multi_logloss: 0.820305
Early stopping, best iteration is:
[57]	valid's multi_logloss: 0.817443
Lvl_0_Pipe_0_Mod_0_LightGBM fitting and predicting completed


[2020-12-03 18:22:31,506] (INFO): Trial 8 finished with value: -0.8174432761780918 and parameters: {'feature_fraction': 0.9162213204002109, 'num_leaves': 53}. Best is trial 1 with value: -0.8157873297296464.


Start fitting Lvl_0_Pipe_0_Mod_0_LightGBM ...
Training until validation scores don't improve for 100 rounds
[100]	valid's multi_logloss: 0.821377
Early stopping, best iteration is:
[52]	valid's multi_logloss: 0.815787
Training until validation scores don't improve for 100 rounds
[100]	valid's multi_logloss: 0.836264
Early stopping, best iteration is:
[47]	valid's multi_logloss: 0.825268
Training until validation scores don't improve for 100 rounds
[100]	valid's multi_logloss: 0.833029
Early stopping, best iteration is:
[28]	valid's multi_logloss: 0.821758
Training until validation scores don't improve for 100 rounds
[100]	valid's multi_logloss: 0.841979
Early stopping, best iteration is:
[33]	valid's multi_logloss: 0.829801
Training until validation scores don't improve for 100 rounds
[100]	valid's multi_logloss: 0.837374
Early stopping, best iteration is:
[20]	valid's multi_logloss: 0.827673
Lvl_0_Pipe_0_Mod_0_LightGBM fitting and predicting completed
Start fitting Lvl_0_Pipe_0_Mod_1_

[2020-12-03 18:23:07,770] (INFO): oof_pred:
array([[0.47555572, 0.02303292, 0.5014114 ],
       [0.41740537, 0.04869818, 0.53389657],
       [0.43521076, 0.03908963, 0.5256996 ],
       ...,
       [0.42236817, 0.01946547, 0.5581664 ],
       [0.4997952 , 0.01693011, 0.48327476],
       [0.38136816, 0.02851641, 0.5901155 ]], dtype=float32)
Shape = (8000, 3)


CPU times: user 4min 50s, sys: 3.19 s, total: 4min 54s
Wall time: 1min 17s


### Predict to test data and check scores

In [18]:
%%time

test_pred = automl.predict(test_data)
logging.debug('Prediction for test data:\n{}\nShape = {}'
              .format(test_pred, test_pred.shape))

logging.info('Check scores...')
logging.info('OOF score: {}'.format(log_loss(train_data[TARGET_NAME].values, oof_pred.data)))
logging.info('TEST score: {}'.format(log_loss(test_data[TARGET_NAME].values, test_pred.data)))

[2020-12-03 18:23:08,475] (INFO): Check scores...
[2020-12-03 18:23:08,479] (INFO): OOF score: 0.8199497586041689
[2020-12-03 18:23:08,480] (INFO): TEST score: 0.8231789521276951


CPU times: user 1.03 s, sys: 65 ms, total: 1.1 s
Wall time: 703 ms


### Check AUCs for each class in train and test data 

In [19]:
for dat, df, name in zip([oof_pred, test_pred], [train_data, test_data], ['train', 'test']):
    logging.debug('Check aucs {0}...'.format(name))
    for cl in range(3):
        sc = roc_auc_score((df[TARGET_NAME].values == cl).astype(np.float32), dat.data[:, cl])
        logging.info('Class {0} {1} auc score: {2}'.format(cl, name, sc))

[2020-12-03 18:23:08,491] (INFO): Class 0 train auc score: 0.5539662631462542
[2020-12-03 18:23:08,495] (INFO): Class 1 train auc score: 0.7196008226136308
[2020-12-03 18:23:08,499] (INFO): Class 2 train auc score: 0.5439895115607825
[2020-12-03 18:23:08,501] (INFO): Class 0 test auc score: 0.5426527076899903
[2020-12-03 18:23:08,503] (INFO): Class 1 test auc score: 0.7214388020833333
[2020-12-03 18:23:08,506] (INFO): Class 2 test auc score: 0.5266755019879473
